In [1]:
import zipfile

In [2]:
# !wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
# zip_ext = zipfile.ZipFile('10_food_classes_10_percent.zip', 'r')
# zip_ext.extractall()
# zip_ext.close()

In [3]:
import os

for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
    print(f'there are {len(dirnames)} directories and {len(filenames)} in {dirpath}')

there are 2 directories and 0 in 10_food_classes_10_percent
there are 10 directories and 0 in 10_food_classes_10_percent/test
there are 0 directories and 250 in 10_food_classes_10_percent/test/ice_cream
there are 0 directories and 250 in 10_food_classes_10_percent/test/chicken_curry
there are 0 directories and 250 in 10_food_classes_10_percent/test/steak
there are 0 directories and 250 in 10_food_classes_10_percent/test/sushi
there are 0 directories and 250 in 10_food_classes_10_percent/test/chicken_wings
there are 0 directories and 250 in 10_food_classes_10_percent/test/grilled_salmon
there are 0 directories and 250 in 10_food_classes_10_percent/test/hamburger
there are 0 directories and 250 in 10_food_classes_10_percent/test/pizza
there are 0 directories and 250 in 10_food_classes_10_percent/test/ramen
there are 0 directories and 250 in 10_food_classes_10_percent/test/fried_rice
there are 10 directories and 0 in 10_food_classes_10_percent/train
there are 0 directories and 75 in 10_fo

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5

train_dir ='10_food_classes_10_percent/train'
test_dir ='10_food_classes_10_percent/test'

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

print('training images:')
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')


print('testing images:')
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

training images:
Found 750 images belonging to 10 classes.
testing images:
Found 2500 images belonging to 10 classes.


In [7]:
import datetime
from tensorflow.keras.callbacks import TensorBoard

def create_tensorflow_callback(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')
    tensorboard_callback = TensorBoard(log_dir=log_dir)
    print(f' saving tensorflow log files to {log_dir}')

    return tensorboard_callback


In [8]:
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'
efficientnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [10]:
def create_model(model_url, num_classes=10):
    """
    Takes tensorflow models from hub and creates model
    """

    feature_extrator_layer = hub.KerasLayer(
        model_url, 
        trainable=False, 
        name='feature_extration_layer',
        input_shape=IMAGE_SHAPE+(3,))


    model = Sequential([
        feature_extrator_layer,
        Dense(num_classes, activation='softmax', name='output_layer')])

    return model


In [11]:
resnet_model = create_model(resnet_url, num_classes=10)

2021-12-20 17:06:34.807865: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extration_layer (Ke  (None, 2048)             23564800  
 rasLayer)                                                       
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [15]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

resnet_model.compile(
    loss=CategoricalCrossentropy(),
    optimizer=Adam(),
    metrics=['accuracy'])



In [16]:
resnet_model.fit(
    train_data, 
    epochs=EPOCHS,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_batch_size=len(test_data),
    callbacks=[create_tensorflow_callback('tensorflow_hub', 'resnet50V2')])

Epoch 1/5
24/24 [==============================] - 343s 14s/step - loss: 2.0195 - accuracy: 0.3427 - val_loss: 1.2342 - val_accuracy: 0.6104
Epoch 2/5
24/24 [==============================] - 272s 12s/step - loss: 0.9427 - accuracy: 0.7173 - val_loss: 0.8739 - val_accuracy: 0.7108
Epoch 3/5
24/24 [==============================] - 256s 11s/step - loss: 0.6424 - accuracy: 0.8120 - val_loss: 0.7580 - val_accuracy: 0.7548
Epoch 4/5
24/24 [==============================] - 243s 10s/step - loss: 0.4902 - accuracy: 0.8640 - val_loss: 0.7193 - val_accuracy: 0.7680
Epoch 5/5
24/24 [==============================] - 314s 14s/step - loss: 0.3945 - accuracy: 0.9067 - val_loss: 0.6850 - val_accuracy: 0.7764


In [17]:
efficientnet_model = create_model(efficientnet_url, num_classes=10)

In [18]:
efficientnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extration_layer (Ke  (None, 1280)             4049564   
 rasLayer)                                                       
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 4,062,374
Trainable params: 12,810
Non-trainable params: 4,049,564
_________________________________________________________________


In [19]:
efficientnet_model.compile(
    loss=CategoricalCrossentropy(),
    optimizer=Adam(),
    metrics=['accuracy'])

In [20]:
efficientnet_model.fit(
    train_data, 
    epochs=EPOCHS,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_batch_size=len(test_data),
    callbacks=[create_tensorflow_callback('tensorflow_hub', 'efficientnet0')])

 saving tensorflow log files to tensorflow_hub/efficientnet0/2021_12_20-17_32_51
Epoch 1/5
24/24 [==============================] - 193s 8s/step - loss: 1.8398 - accuracy: 0.4280 - val_loss: 1.2902 - val_accuracy: 0.7448
Epoch 2/5
24/24 [==============================] - 154s 7s/step - loss: 1.0409 - accuracy: 0.7787 - val_loss: 0.8585 - val_accuracy: 0.8288
Epoch 3/5
24/24 [==============================] - 150s 6s/step - loss: 0.7353 - accuracy: 0.8507 - val_loss: 0.6886 - val_accuracy: 0.8484
Epoch 4/5
24/24 [==============================] - 164s 7s/step - loss: 0.5922 - accuracy: 0.8733 - val_loss: 0.6076 - val_accuracy: 0.8552
Epoch 5/5
24/24 [==============================] - 163s 7s/step - loss: 0.4989 - accuracy: 0.8920 - val_loss: 0.5542 - val_accuracy: 0.8624
